### Value Based Investing
A model that ranks stocks based on key fundamental valuation metrics (P/E, P/B, P/S, EV/EBITDA, EV/GP) to identify undervalued opportunities.

Import the libraries

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import math
from scipy import stats

Load the Stocks List

In [2]:
tickers = pd.read_csv('top50_us_stocks_info.csv')
tickers.head()

,Name,Ticker,Sector,MarketCap,PreviousClose,52WeekHigh,52WeekLow
0,3M Company,MMM,Industrials,87442300928,160.59,172.85,121.98
1,A.O. Smith Corporation,AOS,Industrials,9207796736,65.15,77.92,58.83
2,Abbott Laboratories,ABT,Healthcare,217098158080,125.80,141.23,110.86
3,AbbVie Inc.,ABBV,Healthcare,383705284608,215.89,244.81,163.81
4,Accenture plc,ACN,Technology,153862651904,242.90,398.35,229.40


Fetch Valuation Metrics

In [3]:
# Fetch valuation metrics for each stock: PE, PB, PS, EV/EBITA, EV/GP
def fetch_values_of_stocks(tickers):
    
    # Step 1: Define cols to store stock valuation metrics
    value_cols = [
        'Ticker',
        'Price',
        'PE - Ratio',
        'PB - Ratio',
        'PS - Ratio',
        'EV/EBITDA',
        'EV/GP'
    ]

    # Step 2 : Initialize empty DataFrame
    value_df = pd.DataFrame(columns=value_cols)

    # Step 3: Fetch metrics for each ticker
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        price = stock.history(period='1d')['Close'].iloc[-1]

        financials = stock.financials
        balanceSheet = stock.balance_sheet
        cashFlow = stock.cash_flow

        # Get key valuation ratios from stock info
        pe_ratio = stock.info.get('forwardPE', np.nan)
        pb_ratio = stock.info.get('priceToBook', np.nan)
        ps_ratio = stock.info.get('priceToSalesTrailing12Months', np.nan)
        ev = stock.info.get('enterpriseValue', np.nan)
        ebitda = stock.info.get('ebitda', np.nan)
        evEbitda = ev / ebitda if ev and ebitda else np.nan
        grossProfit = stock.info.get('grossMargins', np.nan) * stock.info.get('totalRevenue', np.nan)
        evGrossProfit = ev / grossProfit if ev and grossProfit else np.nan

        # Get key valuation ratios from stock info
        value_df.loc[len(value_df)] = [
            ticker,
            price,
            pe_ratio,
            pb_ratio,
            ps_ratio,
            evEbitda,
            evGrossProfit
        ]

    return value_df

In [4]:
# Create a list of tickers from the CSV
tickers_list = tickers['Ticker'].values.tolist()
df = fetch_values_of_stocks(tickers_list)
df

,Ticker,Price,PE - Ratio,PB - Ratio,PS - Ratio,EV/EBITDA,EV/GP
0,MMM,163.899994,20.746834,18.813131,3.507262,15.472960,9.370742
1,AOS,65.580002,16.113022,4.950555,2.384068,11.785762,6.266462
2,ABT,124.934998,24.212210,4.263556,4.955089,18.907900,9.004722
3,ABBV,219.104996,18.063066,-146.558530,6.492569,15.120328,10.471937
4,ACN,241.309998,17.150675,4.810326,2.147830,12.389130,6.810145
5,ADBE,327.350006,15.929441,11.681059,5.911251,15.580890,6.831646
6,AMD,237.699997,46.607845,6.357995,12.083101,68.822967,25.085935
7,AES,14.200000,6.995074,2.616547,0.836127,14.883717,22.903442
8,AFL,113.690002,15.856346,2.235396,3.745224,18.896011,9.888498
9,A,147.479996,26.102654,6.565755,6.159493,23.758627,12.235480


Handle Null Values


In [5]:
# Check the DataFrame structure and null values
# Helps identify columns with nulls that may need cleaning or filling
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Ticker      50 non-null     object 
 1   Price       50 non-null     float64
 2   PE - Ratio  50 non-null     float64
 3   PB - Ratio  50 non-null     float64
 4   PS - Ratio  50 non-null     float64
 5   EV/EBITDA   46 non-null     float64
 6   EV/GP       49 non-null     float64
dtypes: float64(6), object(1)
memory usage: 3.1+ KB


In [6]:
# Fill null values in key valuation cols with the col mean
value_cols = [
    'Price',
    'PE - Ratio',
    'PB - Ratio',
    'PS - Ratio',
    'EV/EBITDA',
    'EV/GP'
]

for col in value_cols:
    df[col] = df[col].fillna(df[col].mean())

# Verify that null values have been handled
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 0 to 49
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Ticker      50 non-null     object 
 1   Price       50 non-null     float64
 2   PE - Ratio  50 non-null     float64
 3   PB - Ratio  50 non-null     float64
 4   PS - Ratio  50 non-null     float64
 5   EV/EBITDA   50 non-null     float64
 6   EV/GP       50 non-null     float64
dtypes: float64(6), object(1)
memory usage: 3.1+ KB


Calculate Percentiles

In [7]:
# Calculate percentile rank for each valuation metric
# Percentiles show how a stock compares to others in the dataset
percentile_matrix = {
    'PE - Ratio': 'PE-Ratio_Percentile',
    'PB - Ratio': 'PB-Ratio_Percentile',
    'PS - Ratio': 'PS-Ratio_Percentile',
    'EV/EBITDA': 'EV/EBITDA_Percentile',
    'EV/GP': 'EV/GP_Percentile'
}

for matrix, percentile in percentile_matrix.items():
    df[percentile] = df[matrix].apply(lambda x: stats.percentileofscore(df[matrix], x) / 100)

df.head()

,Ticker,Price,PE - Ratio,PB - Ratio,PS - Ratio,EV/EBITDA,EV/GP,PE-Ratio_Percentile,PB-Ratio_Percentile,PS-Ratio_Percentile,EV/EBITDA_Percentile,EV/GP_Percentile
0,MMM,163.899994,20.746834,18.813131,3.507262,15.472960,9.370742,0.58,0.92,0.42,0.40,0.42
1,AOS,65.580002,16.113022,4.950555,2.384068,11.785762,6.266462,0.42,0.60,0.28,0.18,0.20
2,ABT,124.934998,24.212210,4.263556,4.955089,18.907900,9.004722,0.70,0.52,0.62,0.58,0.38
3,ABBV,219.104996,18.063066,-146.558530,6.492569,15.120328,10.471937,0.46,0.02,0.78,0.38,0.48
4,ACN,241.309998,17.150675,4.810326,2.147830,12.389130,6.810145,0.44,0.58,0.22,0.20,0.26


Compute Value Score

In [8]:
# Calc an overall Value Score for each stock
# Value Score = average of all percentile ranks for PE, PB, PS, EV/EBITA, EV/GP
# Higher score = relatively better valuation compared to peers

df['Value_Score'] = df[(value for value in percentile_matrix.values())].mean(axis = 1)
df

,Ticker,Price,PE - Ratio,PB - Ratio,PS - Ratio,EV/EBITDA,EV/GP,PE-Ratio_Percentile,PB-Ratio_Percentile,PS-Ratio_Percentile,EV/EBITDA_Percentile,EV/GP_Percentile,Value_Score
0,MMM,163.899994,20.746834,18.813131,3.507262,15.472960,9.370742,0.58,0.92,0.42,0.40,0.42,0.548
1,AOS,65.580002,16.113022,4.950555,2.384068,11.785762,6.266462,0.42,0.60,0.28,0.18,0.20,0.336
2,ABT,124.934998,24.212210,4.263556,4.955089,18.907900,9.004722,0.70,0.52,0.62,0.58,0.38,0.560
3,ABBV,219.104996,18.063066,-146.558530,6.492569,15.120328,10.471937,0.46,0.02,0.78,0.38,0.48,0.424
4,ACN,241.309998,17.150675,4.810326,2.147830,12.389130,6.810145,0.44,0.58,0.22,0.20,0.26,0.340
5,ADBE,327.350006,15.929441,11.681059,5.911251,15.580890,6.831646,0.40,0.84,0.68,0.44,0.28,0.528
6,AMD,237.699997,46.607845,6.357995,12.083101,68.822967,25.085935,0.94,0.62,0.96,1.00,0.94,0.892
7,AES,14.200000,6.995074,2.616547,0.836127,14.883717,22.903442,0.02,0.42,0.06,0.36,0.92,0.356
8,AFL,113.690002,15.856346,2.235396,3.745224,18.896011,9.888498,0.38,0.30,0.46,0.56,0.46,0.432
9,A,147.479996,26.102654,6.565755,6.159493,23.758627,12.235480,0.74,0.64,0.72,0.82,0.58,0.700


In [9]:
# Sort stocks in descending order of Value Score
# Higher Value Score indicates a more attractive valuation
df = df.sort_values(by='Value_Score', ascending = False)
df

,Ticker,Price,PE - Ratio,PB - Ratio,PS - Ratio,EV/EBITDA,EV/GP,PE-Ratio_Percentile,PB-Ratio_Percentile,PS-Ratio_Percentile,EV/EBITDA_Percentile,EV/GP_Percentile,Value_Score
40,APP,621.359985,120.418600,193.872070,36.631220,61.517513,46.789461,1.00,1.00,1.00,0.98,1.00,0.996
38,AAPL,269.769989,32.463295,54.051290,9.578544,27.975267,20.744659,0.90,0.98,0.92,0.88,0.88,0.912
33,APH,138.110001,63.939810,13.497849,8.060376,28.741350,22.882317,0.98,0.86,0.84,0.90,0.90,0.896
6,AMD,237.699997,46.607845,6.357995,12.083101,68.822967,25.085935,0.94,0.62,0.96,1.00,0.94,0.892
49,ADSK,294.859985,32.119823,23.133530,9.508732,39.176764,10.627194,0.88,0.96,0.90,0.94,0.50,0.836
20,GOOG,285.339996,31.881565,8.907688,8.924710,23.294647,14.826252,0.86,0.76,0.88,0.80,0.78,0.816
44,ANET,134.020004,13.773896,14.159535,19.976620,45.364826,30.637350,0.30,0.88,0.98,0.96,0.96,0.816
28,AMT,178.039993,26.220913,21.092287,7.972378,19.897039,17.237769,0.76,0.94,0.82,0.70,0.80,0.804
19,GOOGL,284.750000,31.780134,8.889270,8.920038,23.258074,14.802975,0.84,0.74,0.86,0.78,0.76,0.796
34,ADI,232.880005,30.885942,3.361141,11.030023,26.112144,19.421070,0.82,0.48,0.94,0.86,0.86,0.792


Rank and Select Top Stocks

In [10]:
# Keep only the top 10 stocks based on Value Score
df = df.head(10)

# Reset index for a clean 0-9 range
df.reset_index(drop=True, inplace=True)
df

,Ticker,Price,PE - Ratio,PB - Ratio,PS - Ratio,EV/EBITDA,EV/GP,PE-Ratio_Percentile,PB-Ratio_Percentile,PS-Ratio_Percentile,EV/EBITDA_Percentile,EV/GP_Percentile,Value_Score
0,APP,621.359985,120.418600,193.872070,36.631220,61.517513,46.789461,1.00,1.00,1.00,0.98,1.00,0.996
1,AAPL,269.769989,32.463295,54.051290,9.578544,27.975267,20.744659,0.90,0.98,0.92,0.88,0.88,0.912
2,APH,138.110001,63.939810,13.497849,8.060376,28.741350,22.882317,0.98,0.86,0.84,0.90,0.90,0.896
3,AMD,237.699997,46.607845,6.357995,12.083101,68.822967,25.085935,0.94,0.62,0.96,1.00,0.94,0.892
4,ADSK,294.859985,32.119823,23.133530,9.508732,39.176764,10.627194,0.88,0.96,0.90,0.94,0.50,0.836
5,GOOG,285.339996,31.881565,8.907688,8.924710,23.294647,14.826252,0.86,0.76,0.88,0.80,0.78,0.816
6,ANET,134.020004,13.773896,14.159535,19.976620,45.364826,30.637350,0.30,0.88,0.98,0.96,0.96,0.816
7,AMT,178.039993,26.220913,21.092287,7.972378,19.897039,17.237769,0.76,0.94,0.82,0.70,0.80,0.804
8,GOOGL,284.750000,31.780134,8.889270,8.920038,23.258074,14.802975,0.84,0.74,0.86,0.78,0.76,0.796
9,ADI,232.880005,30.885942,3.361141,11.030023,26.112144,19.421070,0.82,0.48,0.94,0.86,0.86,0.792


Determine Position Size

In [ ]:
# Get total investment amount from user
portfolio_size = int(input("Enter the investment amount: "))

# Calculate equal position size per stock
position_size = portfolio_size / len(df.index)
position_size

Calculate Number of Shares

In [ ]:
# Calc number of shares to buy for each stock based on position size
# Use floor to avoid fractional shares
df['Number of Shares to buy'] = df['Price'].apply(lambda price: math.floor( 
    position_size / price
))
df